# DETERMINING BEST NEIGHBORHOODS ALTERNATIVES TO FOUND YOGA STUDIO IN TORONTO

### Required Libraries 

In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup

import folium # map rendering library

from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print("Libraries imported.")

Libraries imported.


## Download Dataset 

To obtain the dataset which includes the **postal code, borough and neighborhood** of Toronto, we will utilize the Wikipedia page and by scraping the table in [this link](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M), we will be able to generate the required dataframe.  
  
Get the html of the wikipedia page with **requests** and **beautifulsoup** libraries. 
  
The required information is in the "tr" and "td" HTML items. Former one shows the rows and latter one denotes the ingredient of that row.

In [2]:
# GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

postalCode=[]
borough=[]
neighborhood=[]

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCode.append(cells[0].text.rstrip('\n'))
        borough.append(cells[1].text.rstrip('\n'))
        neighborhood.append(cells[2].text.rstrip('\n'))

Now, we have 3 lists which contains related information, so we create a dataframe via Pandas and then, get rid of _not assigned_ cells in **Borough** column.

In [14]:
toronto_df=pd.DataFrame({"Postal Code": postalCode,"Borough": borough,"Neighborhood": neighborhood})
toronto_df=toronto_df[toronto_df["Borough"] != "Not assigned"].reset_index(drop=True)
print(toronto_df.shape)
toronto_df.head()

(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
toronto_df["Neighborhood"]=np.where(toronto_df["Neighborhood"]=="Not assigned",toronto_df["Borough"],toronto_df["Neighborhood"])

### **Some exploratory works**

In [11]:
print("There are {} unique boroughs and {} unique neighborhoods in Toronto".format(len(toronto_df["Borough"].unique()),len(toronto_df["Neighborhood"].unique())))

There are 10 unique boroughs and 98 unique neighborhoods in Toronto


In [12]:
toronto_df.isnull().sum()

Postal Code     0
Borough         0
Neighborhood    0
dtype: int64

There is no null value in our dataframe.

In [13]:
print("List of Boroughs with the number of neighborhoods in there\n")
print(toronto_df["Borough"].value_counts())

List of Boroughs with the number of neighborhoods in there

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64


In [17]:
# Group by Borough and Postal Codes by putting "," between Neighborhoods. 
toronto_grouped = toronto_df.groupby(["Postal Code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_grouped.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Now, we will add to coordinates to the above dataframes by using Postal Codes. Here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [15]:
coor= pd.read_csv('https://cocl.us/Geospatial_data')
coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
#merging 2 dfs for final data
toronto_final = toronto_grouped.merge(coor, on="Postal Code", how="left")
toronto_final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ozcan_app")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Borough'], toronto_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [21]:
#Foursquare API credentials
CLIENT_ID = '5H1IIXS3HEMFQIRSVHKDVU40MKVBKIT0JBDSIELPQE41SZCC' # your Foursquare ID
CLIENT_SECRET = 'O20SAVKDFFZVX1ZZKUTF1JYVXLQNQSVL05EN1NFKYC1B1ZM4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5H1IIXS3HEMFQIRSVHKDVU40MKVBKIT0JBDSIELPQE41SZCC
CLIENT_SECRET:O20SAVKDFFZVX1ZZKUTF1JYVXLQNQSVL05EN1NFKYC1B1ZM4


In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [25]:
toronto_venues = getNearbyVenues(names=toronto_final['Neighborhood'],latitudes=toronto_final['Latitude'],longitudes=toronto_final['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence P

In [50]:
print("There are {} venues in Toronto distributed in {} distinct neighborhoods".format(toronto_venues.shape[0],
                                                                                       len(toronto_venues["Neighborhood"].unique())))

There are 2119 venues in Toronto distributed in 93 distinct neighborhoods


In [46]:
#to check wheter there are Yoga Studio venues in Toronto
"Yoga Studio" in toronto_venues['Venue Category'].unique()

True

In [48]:
print("There are {} yoga studios in Toronto".format(len(toronto_venues[toronto_venues['Venue Category']=="Yoga Studio"])))

There are 14 yoga studios in Toronto


In [58]:
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(2119, 264)


,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
to_grouped = to_onehot.groupby(["Neighborhood"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(93, 264)


,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.052632,0.000000,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,"Wexford, Maryvale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
89,Willowdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.025641,0.0,0.0,0.0,0.0
90,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
91,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.111111,0.000000,0.0,0.0,0.0,0.0


In [68]:
from sklearn.cluster import KMeans
toclusters = 5

to_clustering = to_grouped.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 0, 4, 0,
       4, 0, 1, 0, 0, 2, 1, 0, 2, 0, 1, 0, 0, 4, 1, 0, 4, 1, 4, 0, 0, 1,
       0, 3, 0, 4, 2, 4, 0, 0, 4, 0, 0, 4, 0, 4, 0, 1, 4, 0, 0, 4, 4, 1,
       0, 4, 4, 0, 1])

In [69]:
to_grouped["Cluster"]=kmeans.labels_
to_grouped.sample(5)

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Cluster
30,Forest Hill North & West,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
87,Weston,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
59,"Parkdale, Roncesvalles",0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,Business reply mail Processing Centre,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
86,Westmount,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [70]:
merged = to_grouped.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")
merged.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Wine Bar,Wings Joint,Women's Store,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.7942,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.7942,-79.262029,El Pulgarcito,43.792648,-79.259208,Latin American Restaurant
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.7942,-79.262029,Twilight,43.791999,-79.258584,Lounge
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.7942,-79.262029,Mark's,43.791179,-79.259714,Clothing Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,43.7942,-79.262029,Commander Arena,43.794867,-79.267989,Skating Rink


In [71]:
#Cluster 0
merged.loc[(merged['Cluster'] ==0) & (merged['Venue Category'] == 'Yoga Studio') ]

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Wine Bar,Wings Joint,Women's Store,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
8,Business reply mail Processing Centre,0.062500,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0,43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
15,Church and Wellesley,0.025974,0.0,0.012987,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.012987,0.0,0,43.665860,-79.383160,Bikram Yoga Yonge,43.668205,-79.385780,Yoga Studio
15,Church and Wellesley,0.025974,0.0,0.012987,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.012987,0.0,0,43.665860,-79.383160,The Yoga Sanctuary,43.661499,-79.383636,Yoga Studio
48,"Little Portugal, Trinity",0.024390,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.02439,0.000000,0.0,0,43.647927,-79.419750,YogaSpace,43.647607,-79.420133,Yoga Studio
55,North Toronto West,0.045455,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0,43.715383,-79.405678,Barreworks,43.714070,-79.400109,Yoga Studio
63,"Regent Park, Harbourfront",0.021739,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0,43.654260,-79.360636,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
68,"Runnymede, Swansea",0.026316,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0,43.651571,-79.484450,(The New) Moksha Yoga Bloor West,43.648658,-79.485242,Yoga Studio
75,Stn A PO Boxes,0.010638,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0,43.646435,-79.374846,Bikram Yoga Centre,43.649214,-79.375229,Yoga Studio
76,Studio District,0.025000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.02500,0.000000,0.0,0,43.659526,-79.340923,Spirit Loft Yoga,43.663548,-79.341333,Yoga Studio
80,"The Danforth West, Riverdale",0.023810,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0,43.679557,-79.352188,Moksha Yoga Danforth,43.677622,-79.352116,Yoga Studio


In [72]:
#Cluster 1
merged.loc[(merged['Cluster'] ==1) & (merged['Venue Category'] == 'Yoga Studio') ]

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Wine Bar,Wings Joint,Women's Store,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [73]:
#Cluster 2
merged.loc[(merged['Cluster'] ==2) & (merged['Venue Category'] == 'Yoga Studio') ]

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Wine Bar,Wings Joint,Women's Store,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [74]:
#Cluster 3
merged.loc[(merged['Cluster'] ==3) & (merged['Venue Category'] == 'Yoga Studio') ]

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Wine Bar,Wings Joint,Women's Store,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [75]:
#Cluster 4
merged.loc[(merged['Cluster'] ==4) & (merged['Venue Category'] == 'Yoga Studio') ]

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Wine Bar,Wings Joint,Women's Store,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
13,Central Bay Street,0.016393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,4,43.657952,-79.387383,The Yoga Sanctuary,43.661499,-79.383636,Yoga Studio
62,"Queen's Park, Ontario Provincial Government",0.029412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.029412,0.0,4,43.662301,-79.389494,The Yoga Sanctuary,43.661499,-79.383636,Yoga Studio
82,Thorncliffe Park,0.050000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,4,43.705369,-79.349372,Bikram Yoga East York,43.705450,-79.351448,Yoga Studio


In [80]:
yoga_studio_df=merged[merged['Venue Category'] == 'Yoga Studio']
yoga_studio_df["Cluster"].value_counts()

0    11
4     3
Name: Cluster, dtype: int64

# CONCLUSION

As seen, when we divide Toronto into 5 clusters based on the all venues, we can see that cluster 1,2 and 3 do not have any yoga studio. Therefore, these neighborhoods can be great alternatives to run yoga studio.